In [26]:
from arcgis.gis import GIS
gis = GIS("home", verify_cert=False)

/opt/conda/lib/python3.6/site-packages/arcgis/gis/__init__.py:407: UserWarning: You are logged on as gregbrunner_slugis with an administrator role, proceed with caution.
  self.users.me.username)


# Intro to Arcpy

In these excercises, we will start working with the Arcpy site package, which allows access to ArcGIS processing and analysis capabilities through Python. Arcpy contains modules, classes, and functions, all of which will give you access to capabliities that you should already be familiar with in ArcGIS.

## Arcpy

When developing a script in Python that will access an ArcGIS geoprocessing tool, the first thing that we must do is import arcpy.

In [1]:
import arcpy

After importing arcpy, we can investigate it's properties such as the path and version.

In [2]:
print(arcpy.__path__)

['/opt/conda/lib/python3.6/site-packages/arcpy']


In [3]:
arcpy.GetInstallInfo()

{'LicenseLevel': 'Advanced',
 'InstallDir': 'z:\\home\\arcgis\\arcgis\\server\\framework\\runtime\\arcgis\\',
 'Installer': 'bldgis',
 'ProductName': 'Server',
 'Version': '10.8.1',
 'SourceDir': 'Z:\\mnt\\setupteam\\10.8.1\\Server\\SetupFiles\\',
 'InstallType': 'N/A',
 'BuildNumber': '24234',
 'InstallDate': '6/15/2020',
 'InstallTime': '11:53:50',
 'SPNumber': 'N/A',
 'SPBuild': 'N/A',
 'ProVersion': '2.6.0'}

Often, when writing a script, the first this you do is set the geoprocessing environment default workspace.  This can be either a geodatabase of folder and can be done as follows:

In [5]:
import os

os.listdir('home')

['.nb_auth_file', '.ipynb_checkpoints', 'GISData', '.snapshot']

In [6]:
arcpy.env.workspace = "/arcgis/home/GISData"
print(arcpy.env.workspace)

/arcgis/home/GISData


Sometimes, you may not want to import all of arcpy. You can set the _env_ variables without importing all of arcpy using a _from_-_import_ statment as follows:

In [7]:
from arcpy import env

Then, you could set the default workspace as follows:

In [8]:
env.workspace =  "/arcgis/home/GISData"
print(env.workspace)

/arcgis/home/GISData


Not to confuse you, but there is a third way to import the env variable, using _from_-_import_-_as_. Here is an example:

In [9]:
from arcpy import env as myenv

Using this syntax, we can set the workspace as follows:

In [10]:
myenv.workspace = "/arcgis/home/GISData"
print(myenv.workspace)

/arcgis/home/GISData


## Arcpy.mapping and Spatial Analyst

Two modules that we will use later in this course are arcpy.mapping and spatial analyst. These modules allow users to carry out specialized functionality using ArcGIS. Arcpy.mapping is imported as follows:

In [12]:
import arcpy.mp

You will typically see spatial analyst imported as follows:

In [13]:
from arcpy.sa import *

This import statement uses the wildcard to import all of the functions from the spatial analyst module.  You will get a better understanding of this syntax and the spatial analyst module when we start working with raster/imagery data.

# Geoprocessing Tools

## Clip

The following excercises will show us how to work with Geoprocessing tools through Python. You saw some of this last week when we used the [Clip function](http://pro.arcgis.com/en/pro-app/tool-reference/analysis/clip.htm) in the Python Window. We will build on that here. First, let's write a short script that takes in two input feature classes and clips one to the other. Here, we will use the same sample data in the Week_1\data folder that we used last week.

In [22]:
## Sample script to that uses the clip features GP tool.
import arcpy

input_fc = '/arcgis/home/GISData/Missouri.gdb/Roads'
clip_fc = '/arcgis/home/GISData/Missouri.gdb/Saint_Charles'
output_fc = '/arcgis/home/GISData/Missouri.gdb/Saint_Charles_roads'
#Note: the Geodatabase needs to exist beforehand!

arcpy.Clip_analysis(input_fc, clip_fc, output_fc)

<Result '/arcgis/home/GISData/Missouri.gdb/Saint_Charles_roads'>

### Check to see if it is in the GDB

In [23]:
env.workspace =  "/arcgis/home/GISData/Missouri.gdb"
print(env.workspace)

/arcgis/home/GISData/Missouri.gdb


In [32]:
arcpy.ListFeatureClasses()

['County', 'Saint_Charles', 'Roads', 'Saint_Charles_roads']

### Let's view this data on a map

In [19]:
import pandas as pd

In [27]:
sedf = pd.DataFrame.spatial.from_featureclass('Saint_Charles_roads')
sedf.spatial.project(spatial_reference = {'wkid': 3857})
sedf.head()

,OBJECTID_1,OBJECTID,highway,building,natural,waterway,amenity,landuse,place,railway,...,osmuser,osmuid,osmvisible,osmversion,osmchanges,osmtimesta,osmSupport,osm_highwa,SHAPE_Leng,SHAPE
0,1,3092,trunk,,,,,,,,...,maxerickson,360392,true,18,32600463,2015-07-13,no,,0.000910,"{""paths"": [[[-90.91847019963996, 38.8875270998..."
1,2,7511,secondary,,,,,,,,...,Mark Sims,382726,true,8,13355620,2012-10-03,no,,0.002388,"{""paths"": [[[-90.7503962001245, 38.92244730010..."
2,3,7801,residential,,,,,,,,...,BCNorwich,322785,true,5,14034280,2012-11-25,no,,0.000676,"{""paths"": [[[-90.59011730017323, 38.7809737997..."
3,4,7802,residential,,,,,,,,...,bot-mode,451693,true,2,15792274,2013-04-19,no,,0.000566,"{""paths"": [[[-90.59945799978681, 38.7719174998..."
4,5,7803,residential,,,,,,,,...,CmdrThor,1227748,true,2,15013816,2013-02-13,no,,0.001973,"{""paths"": [[[-90.6740001997494, 38.79305129997..."


In [28]:
mo_map = gis.map('St. Charles, MO')
mo_map

MapView(layout=Layout(height='400px', width='100%'))

In [31]:
sedf.spatial.plot(mo_map, 
                alpha=0.75, 
                line_width = 0.5) 

True

### Delete Roads

In [51]:
arcpy.Delete_management('Saint_Charles_roads')

<Result 'true'>

## Buffer the Features

Another common geoprocessing task if creating a [buffer](https://pro.arcgis.com/en/pro-app/tool-reference/analysis/buffer.htm). Let's show how that's done with Python

In [33]:
## Sample script to that uses the buffer GP tool.
input_fc = 'Saint_Charles'
output_fc = 'Saint_Charles_buffer'
#Note: the Geodatabase needs to exist beforehand!

arcpy.Buffer_analysis(input_fc, output_fc, "10 kilometers", "FULL", "ROUND")

<Result '/arcgis/home/GISData/Missouri.gdb/Saint_Charles_buffer'>

## Question 2: Did buffer work? Go to ArcMap and check. If it did not work, what was the error message you recieved?

In [34]:
mo_map_2 = gis.map('Missouri, USA')
mo_map_2

MapView(layout=Layout(height='400px', width='100%'))

In [35]:
stc_sedf = pd.DataFrame.spatial.from_featureclass('Saint_Charles_buffer')
stc_sedf.spatial.project(spatial_reference = {'wkid': 3857})
stc_sedf.head()

,OBJECTID,COUNTYNS,GEOID,NAMELSAD,CLASSFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BUFF_DIST,ORIG_FID,SHAPE
0,1,00758546,29183,St. Charles County,H1,A,1.451532e+09,83314211.0,+38.7811023,-90.6749153,10.0,1,"{""rings"": [[[-90.44116561699997, 39.0575175640..."


In [36]:
stc_sedf.spatial.plot(mo_map_2, 
                alpha=0.75, 
                line_width = 0.5) 

True

### Delete Buffer

In [52]:
arcpy.Delete_management('Saint_Charles_buffer')

<Result 'true'>

## Count Features with GetCount_management()

Lastly, we'll revisit the [GetCount_management() tool](https://pro.arcgis.com/en/pro-app/tool-reference/data-management/get-count.htm). Let's write a script that Gets the count of all of the features in out Missouri.gdb database. Here is how we do that.

In [53]:
## Sample script to that uses the buffer GP tool.
import arcpy

arcpy.env.workspace = '/arcgis/home/GISData/Missouri.gdb'

feature_classes = arcpy.ListFeatureClasses()

for fc in feature_classes:
    fc_count = arcpy.GetCount_management(fc)
    print(fc + ' has ' + str(fc_count) + ' features')

County has 115 features
Saint_Charles has 1 features
Roads has 82745 features


## Question 3: How many features does each featur class in the Missouri.gdb contain?

We will be using these GP tools a lot, so make sure you are comfortable using them and using the Help documentation in order to understand how they are used.

## Functions

Arcpy functions generally fall into the folloing categories:
- Cursors
- Descrobe objects
- Environment variables and settings
- Fields
- Geodatabase administration
- Getting a setting parameters
- Licensing
- Installation info
- Logs, messages, and dialog boxes
- Rasters
- Geoprocessing tools and toolboxes

Here is a complete list of all [arcpy functions](http://pro.arcgis.com/en/pro-app/arcpy/functions/alphabetical-list-of-arcpy-functions.htm).  

Let's look at the [Exists function](https://pro.arcgis.com/en/pro-app/arcpy/functions/exists.htm).

In [39]:
import arcpy
arcpy.Exists("Saint_Charles_roads")

True

In [41]:
arcpy.Exists("Missouri")

False

Next week, in addition to writing our fist scripts, I hope to cover Describe objects.

## Classes

Arcpy classes are generally shortcuts for tool parameters. For example, getting and setting spatial reference information or setting the default workspace using the environment variable like in the examples above.  Here, we'll show how to use the arcpy SpatialReference class. For a more detailed discussion of classes, see chapter 5.8 of Zandbergen or see the [Arcpy help](http://pro.arcgis.com/en/pro-app/arcpy/classes/spatialreference.htm).

In [42]:
sr = arcpy.SpatialReference("Hawaii Albers Equal Area Conic")

In [43]:
print(sr.name)

Hawaii_Albers_Equal_Area_Conic


Let's look at some of the propoerties of the spatial reference object (_sr_).

In [44]:
sr.metersPerUnit

1.0

In [45]:
sr.XYTolerance

0.001

In [46]:
sr.remarks #what are remarks?

''

For a complete list of arcpy classes and a description of each, go [here](http://pro.arcgis.com/en/pro-app/arcpy/classes/alphabetical-list-of-arcpy-classes.htm).

## Messages

I'm not going to say too much about [_Messages_](http://pro.arcgis.com/en/pro-app/arcpy/functions/adderror.htm) in arcpy, other than, in my experience, they work as print statements and debug statements.  When writing a tool, messages work as print statements. So if you plan on creating a GP script/tool, rather than use:

In [48]:
print("Clipping...")

Clipping...


use:

In [49]:
arcpy.AddMessage("Clipping...")

To see an example, let's look at the Arcpy help for [AddMessage](http://pro.arcgis.com/en/pro-app/arcpy/functions/addmessage.htm)

## Licensing and Checking Out Extensions

At times, you will find that you need to use a specific extension from ArcGIS in a Python script. You may find that a script or tool fails becuase it can not check out a specific license.  In those situations, you can use arcpy to check out the license.

In [50]:
import arcpy
if arcpy.CheckExtension("3D") == "Available":
    print('3D Analyst is Available')
    arcpy.CheckOutExtension('3D')
    print('3D Analyst has been checked out')

3D Analyst is Available
3D Analyst has been checked out


For more information on licensing and extensions in ArcGIS, go [here](http://desktop.arcgis.com/en/arcmap/latest/analyze/python/access-to-licensing-and-extensions.htm).

To see a list of extensions and how to check them out with arcpy, go [here](http://desktop.arcgis.com/en/arcmap/latest/analyze/arcpy-functions/checkoutextension.htm).

## Help

Never be afraid to read the help documentation! Learning how to code can be challanging and that is why the help exists. Here are a few links and appraoches that you should use when seeking help.
 
 - [ArcGIS/ArcPy Help](http://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy/what-is-arcpy-.htm): ArcGIS has Python  documentation for every tool, function, class, etc. Learn how to use their examples.
 - Google: Don't be afraid to ask google! It can save you time and give you the code snippets you are looking for.
 - [Stack Overflow](http://stackoverflow.com/): Like google, but for coding problems.
